In [1]:
#42d   socket02s.py
#！/usr/bin/env python3
#coding:utf -8
from pynq.lib.video import *
import cv2
import socket
import re
import time
from math import *
from pynq import Overlay
from pynq.lib import AxiGPIO
ol = Overlay("base.bit")
led_ip = ol.ip_dict['leds_gpio']
switches_ip = ol.ip_dict['switches_gpio']
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
leds = AxiGPIO(led_ip).channel1
s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.bind(('192.168.2.99',8088))
s.listen(1)
print('Waiting for connecting...')
mask = 0xffffffff
leds.write(15, mask)#灯全亮表明在连接
time.sleep(5)

switches = AxiGPIO(switches_ip).channel1

(conn,addr)=s.accept()
print('conn=',conn)
print('addr=',addr)

t1=[]
t2=[]
r=[]
x='1'
while True:
    str1=conn.recv(1024)#发来请求信号1
    
    str2=str(str1,encoding = 'utf - 8')
    
    mask = 0xffffffff
    n=int(str2)
    leds.write(n, mask)#灯变为1，表明接收到请求信号
    #time.sleep(30)
    #print('I received a string is:',str2)
    #str3=str2.upper()
    switches.setdirection("in")
    switches.setlength(3)
    str3=str(switches.read())
    if int(switches.read())==3:
        conn.send(str3.encode('utf - 8'))#为3，不应答
        continue
    else:
        conn.send(str3.encode('utf - 8'))
    conn.send(x.encode('utf - 8'))
    while 1:
        str4=conn.recv(1024)
        #i=i+1
        #str4=str4.decode()
        if not str4:
            break
        #str5=str(str4,encoding = 'utf - 8')
        str4=str4.decode()
        xyr=re.findall("\d+",str4)
        if len(xyr)==3 :
            t1.append(int(xyr[0])*(pi/180))
            t2.append(int(xyr[1])*(pi/180))
        
            r.append(int(xyr[2]))
    
            
        #r1=int(xyr[2])
        conn.send(x.encode('utf-8'))
        
        #print(xyr)
    x=[]
    y=[]
    z=[]
    x=r*np.cos(t1)*np.cos(t2)
    y=r*np.cos(t1)*np.sin(t2)
    
    z=r*np.sin(t1)
    print(x)
    fig = plt.figure() 
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z)
   
    plt.savefig('./new_img.png')
    
    break

conn.close()
s.close()
print('end')
hdmi_out = ol.video.hdmi_out
#初始化HDMI输出
Mode = VideoMode(640,480,24) #原来第三个参数为24

hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
# monitor (output) frame buffer size 
frame_out_w = 1920 
frame_out_h = 1080           #？
'''
fig = plt.figure() 
ax = fig.add_subplot(111, projection='3d')

#ax.plot_surface(x, y, z)
ax.plot_trisurf(x, y, z)
#ax.scatter(x,y,z)
#ax.plot(x,y,z)

plt.savefig('./new_img.png')
'''
from PIL import Image


def IsValidImage(img_path):
    """
    判断文件是否为有效（完整）的图片
    :param img_path:图片路径
    :return:True：有效 False：无效
    """
    bValid = True
    try:
        Image.open(img_path).verify()
    except:
        bValid = False
    return bValid


def transimg(img_path):
    """
    转换图片格式
    :param img_path:图片路径
    :return: True：成功 False：失败
    """
    if IsValidImage(img_path):
        try:
            str = img_path.rsplit(".", 1)
            output_img_path = str[0] + ".jpg"
            print(output_img_path)
            im = Image.open(img_path)
            im=im.convert('RGB')
            im.save(output_img_path)
            return True
        except:
            return False
    else:
        return False
    
print(transimg('./new_img.png'))    
infile='./new_img.jpg'
img=Image.open(infile)
out=img.resize((640,480),Image.ANTIALIAS)
out.save('./new_img.jpg')

dirfile='./new_img.jpg'

img=cv2.imread(dirfile)
size=img.shape
print(size)

##########################################
import matplotlib.image as mpimg # mpimg 用于读取图片
lena = mpimg.imread('new_img.jpg') # 读取和代码处于同一目录下的 lena.png

##############################################图像输出
outframe=hdmi_out.newframe()
outframe[0:480,0:640,:] = lena [0:480,0:640,:]
leds.write(0xf, mask)
hdmi_out.writeframe(outframe) #?

Waiting for connecting...
conn= <socket.socket fd=71, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.2.99', 8088), raddr=('192.168.2.21', 39878)>
addr= ('192.168.2.21', 39878)
[  45.63695384  -30.57195911 -121.35894404 ...,  -60.73204987  -62.29259806
  -60.20163181]
end
./new_img.jpg
True
(480, 640, 3)
